## Ejercicio 7: Uso de N-shot Learning para clasificar correos

En este ejercicio, proporcionamos ejemplos (N-shot) al modelo para mejorar la coherencia en la clasificación de correos de clientes.

In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
API_VERSION = userdata.get('OPENAI_API_VERSION')
AZURE_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')

#print(type(OPENAI_API_KEY),OPENAI_API_KEY)
#print(type(API_VERSION),API_VERSION)
#print(type(AZURE_ENDPOINT),AZURE_ENDPOINT)

In [2]:
from openai import AzureOpenAI

##### Establecemos el valor de la API key de OpenAI. En una aplicación real, no podemos exponer esta API key en el código, y la cogeríamos con una variable de entorno mediante un os.getenv()

##### Definimos ejemplos previos (N-shots)

Algunos de ellos implican una categorización doble. Al restringir las posibles categorías e indicarle que hay casos donde puede haber dos clases a la vez, el modelo clasificará mejor los correos

In [3]:

n_shot_examples = """\
Ejemplo 1:
Correo del cliente:
"Asunto: Cargo duplicado
Hola, creo que hay un cargo duplicado en mi cuenta. ¿Cómo puedo solucionarlo?"
Clasificación: Facturación

Ejemplo 2:
Correo del cliente:
"Asunto: Cambio de domicilio
Buenas tardes, quiero actualizar mi dirección y número de teléfono en mi cuenta bancaria. ¿Cómo debo proceder?"
Clasificación: Actualización de datos

Ejemplo 3:
Correo del cliente:
"Asunto: Error en transferencia
Al intentar hacer una transferencia, me da un error en la app. ¿Pueden ayudarme?"
Clasificación: Problema con transferencia, Problema con la App

Ejemplo 4:
Correo del cliente:
"Asunto: Problema con una transferencia
Ayer intenté hacer una transferencia, pero me pone que "su cuenta ha sido bloqueada y no permite operaciones". ¿Pueden verificar qué ocurre?"
Clasificación: Problema con transferencia, Problema con cuenta bloqueada
"""

##### Tenemos un nuevo email que clasificar

In [4]:
email_cliente = """\
Asunto: Solicitud de préstamo y actualización de ingresos
Quisiera solicitar un préstamo personal, pero para eso necesito actualizar mis datos de ingresos en mi perfil bancario.
¿Me pueden indicar cómo hacerlo?
"""

##### Construimos el prompt añadiendo esos ejemplos

In [5]:
prompt_n_shot = f"""
Eres un asistente bancario especializado en procesar y clasificar correos de clientes según su contenido.
Tu tarea es analizar el contenido de un correo y asignarle **una o más categorías** basándote en ejemplos previos.

A continuación, se presentan **varios ejemplos** de correos reales de clientes, junto con la **clasificación correspondiente**.
**Importante**:
- Si un correo menciona **más de un tema relevante**, debe recibir **más de una categoría**.
- No expliques la clasificación ni des detalles adicionales, **solo responde con la(s) categoría(s) exacta(s)** en formato de texto.
- Si el correo no encaja claramente en una categoría existente, elige la que más se aproxime.

Ejemplos de correos clasificados:

{n_shot_examples}

Ahora clasifica el siguiente correo:
Correo del cliente:
"{email_cliente}"
Clasificación:
"""

##### Llamamos al modelo para ver el resultado

In [6]:
client = AzureOpenAI(
            api_key = OPENAI_API_KEY,
            api_version=API_VERSION,
            azure_endpoint= AZURE_ENDPOINT# Este endpoint corresponde a la región france-central
)

response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_n_shot}],
        temperature=0.1, #Uso una temperatura baja para que no sea creativo, porque me interesa que clasifique lo mejor posible.
        max_tokens=30,
    )

clasificacion_generada = response.choices[0].message.content.strip()
print("\n--- Clasificación Generada ---\n")
print(clasificacion_generada)


--- Clasificación Generada ---

Solicitud de préstamo, Actualización de datos


##### - Cómo cambia la respuesta si modificamos la temperatura?
##### - Qué pasa si le ponemos un correo de un tema que no tiene nada que ver con los ejemplos?